In [9]:
'''Markowitz Optimization in python, using pandas, and Yahoo Finance Data. Input a 
list of tickers, and the program will calcuate historical returns, a covariance matrix, and 
finally, the individual stock weigthings that maximize the portfolio sharpe ratio, defined as
sharpe ratio = (expected portfolio return - risk free rate)/portfolio standard deviation'''



#various pandas, numpy
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats
import matplotlib.mlab as mlab
# plotting

import matplotlib.pyplot as plt

# make plots inline
%matplotlib inline

# formatting options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 10) 
pd.set_option('display.width', 82) 
pd.set_option('precision', 7)

In [10]:
'''Create the function that stores Tickers in a dataframe'''
def create_portfolio(tickers, weights=None):
    if weights is None: 
        shares = np.ones(len(tickers))/len(tickers)
    portfolio = pd.DataFrame({'Tickers': tickers, 
                              'Weights': weights}, 
                             index=tickers)
    return portfolio

In [11]:
def get_historical_closes(ticker, start_date, end_date):
    # get the data for the tickers.  This will be a panel
    p = web.DataReader(ticker, 'yahoo', start_date, end_date)    
    # convert the panel to a DataFrame and selection only Adj Close
    # while making all index levels columns
    d = p.to_frame()['Adj Close'].reset_index()
    # rename the columns
    d.rename(columns={'minor': 'Ticker', 
                      'Adj Close': 'Close'}, inplace=True)
    # pivot each ticker to a column
    pivoted = d.pivot(index='Date', columns='Ticker')
    # and drop the one level on the columns
    pivoted.columns = pivoted.columns.droplevel(0)
    return pivoted

In [ ]:
start = datetime(2015, 1, 1) # Default: Jan 1, 2010
end = datetime(2016, 12, 31) # Default: today

closes = get_historical_closes(['MSFT', 'AAPL', 'KO', 'JPM', 'BRK-B', 'T'], start, end)
closes[:5]

In [ ]:
def calc_daily_returns(closes):
    return np.log(closes/closes.shift(1))

In [ ]:
daily_returns = calc_daily_returns(closes)
daily_returns[:5]

In [ ]:
# calculate annual returns
def calc_annual_returns(daily_returns):
    grouped = np.exp(daily_returns.groupby(
        lambda date: date.year).sum())-1
    return grouped

In [ ]:
annual_returns = calc_annual_returns(daily_returns)
annual_returns

In [ ]:
def calc_portfolio_var(returns, weights=None):
    if weights is None: 
        weights = np.ones(returns.columns.size) / \
        returns.columns.size
    sigma = np.cov(returns.T,ddof=0)
    var = (weights * sigma * weights.T).sum()
    return var

In [ ]:
# calculate our portfolio variance (equal weighted)
calc_portfolio_var(annual_returns)

In [ ]:
def sharpe_ratio(returns, weights = None, risk_free_rate = 0.015):
    n = returns.columns.size
    if weights is None: weights = np.ones(n)/n
    # get the portfolio variance
    var = calc_portfolio_var(returns, weights)
    # and the means of the stocks in the portfolio
    means = returns.mean()
    # and return the sharpe ratio
    return (means.dot(weights) - risk_free_rate)/np.sqrt(var)

In [ ]:
# calculate equal weighted sharpe ratio
sharpe_ratio(annual_returns)

In [ ]:
# function to minimize
def y_f(x): return 2+x**2

In [ ]:
scopt.fmin(y_f,1000)

In [ ]:
def negative_sharpe_ratio_n_minus_1_stock(weights, 
                                          returns, 
                                          risk_free_rate):
    """
    Given n-1 weights, return a negative sharpe ratio
    """
    weights2 = sp.append(weights, 1-np.sum(weights))
    return -sharpe_ratio(returns, weights2, risk_free_rate)

In [ ]:
def optimize_portfolio(returns, risk_free_rate):
    """ 
    Performs the optimization
    """
    # start with equal weights
    w0 = np.ones(returns.columns.size-1, 
                 dtype=float) * 1.0 / returns.columns.size
    # minimize the negative sharpe value
    w1 = scopt.fmin(negative_sharpe_ratio_n_minus_1_stock, 
                    w0, args=(returns, risk_free_rate))
    # build final set of weights
    final_w = sp.append(w1, 1 - np.sum(w1))
    # and calculate the final, optimized, sharpe ratio
    final_sharpe = sharpe_ratio(returns, final_w, risk_free_rate)
    return (final_w, final_sharpe)

In [ ]:
# optimize our portfolio
optimize_portfolio(annual_returns, 0.015)